# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json


# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,linxia chengguanzhen,35.6003,103.2064,11.51,25,0,0.78,CN,1712855579
1,1,bikenibeu village,1.3673,173.1241,27.89,80,100,3.02,KI,1712855579
2,2,pontes e lacerda,-15.2261,-59.3353,25.47,95,100,1.38,BR,1712855579
3,3,vorgashor,67.5833,63.9500,-12.33,82,100,11.70,RU,1712855579
4,4,dudinka,69.4058,86.1778,-25.21,100,55,2.91,RU,1712855579


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width=650,
    frame_height=500,
    size="Humidity",
    color="City"
    )

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_df = city_data_df.loc[(city_data_df["Max Temp"] < 20)
                                       & (city_data_df["Max Temp"] > 10) 
                                       & (city_data_df["Cloudiness"] < 3)
                                       & (city_data_df["Wind Speed"] < 4.5)
                                       &(city_data_df['Country']!='US')
                                      ]

# Drop any rows with null values
ideal_df.dropna()

# Display sample data
ideal_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,linxia chengguanzhen,35.6003,103.2064,11.51,25,0,0.78,CN,1712855579
23,23,lebu,-37.6167,-73.6500,15.47,56,0,2.85,CL,1712855581
30,30,polygyros,40.3744,23.4383,18.75,57,0,2.24,GR,1712855582
36,36,ancud,-41.8697,-73.8203,14.41,53,0,2.14,CL,1712855582
45,45,constantia,44.1833,28.6500,18.38,70,1,1.50,RO,1712855463


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_df.iloc[:,[1,8,2,3,5]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name']= ''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,linxia chengguanzhen,CN,35.6003,103.2064,25,
23,lebu,CL,-37.6167,-73.6500,56,
30,polygyros,GR,40.3744,23.4383,57,
36,ancud,CL,-41.8697,-73.8203,53,
45,constantia,RO,44.1833,28.6500,70,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000 
params = {"categories": "accommodation.hotel",
          "apiKey":geoapify_key
         }

# Print a message to follow up the hotel search
print("Starting hotel search...")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row["Lng"]
    latitude = row["Lat"]

    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] =  f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "[Sorry, No hotel found.]"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - The nearest hotel is: {hotel_df.loc[index, 'Hotel Name']}")
    
# Display sample data
hotel_df

Starting hotel search...
linxia chengguanzhen - The nearest hotel is: 如家酒店
lebu - The nearest hotel is: [Sorry, No hotel found.]
polygyros - The nearest hotel is: Ξενώνας Μαρέλια
ancud - The nearest hotel is: Cabañas
constantia - The nearest hotel is: Hotel Chérica
puerto natales - The nearest hotel is: Dorotea Patagonia Hostel
alexandria - The nearest hotel is: Smoha Zahran Haus (7th floor)
saint-francois - The nearest hotel is: Chez Lily
luc-la-primaube - The nearest hotel is: Le Bistro
sasolburg - The nearest hotel is: Be My Guest Hotel
dzialoszyn - The nearest hotel is: [Sorry, No hotel found.]
serhiyivka - The nearest hotel is: Пансіонат "Калина"
mount isa - The nearest hotel is: Ibis Styles
murray bridge - The nearest hotel is: Bridgeport
puerto deseado - The nearest hotel is: Los Olmos
chonchi - The nearest hotel is: Hotel Huildín
dubbo - The nearest hotel is: Blue Gum Motel
zlatitsa - The nearest hotel is: Комплекс Мутафчиев
charters towers - The nearest hotel is: [Sorry, No ho

,City,Country,Lat,Lng,Humidity,Hotel Name
0,linxia chengguanzhen,CN,35.6003,103.2064,25,如家酒店
23,lebu,CL,-37.6167,-73.6500,56,"[Sorry, No hotel found.]"
30,polygyros,GR,40.3744,23.4383,57,Ξενώνας Μαρέλια
36,ancud,CL,-41.8697,-73.8203,53,Cabañas
45,constantia,RO,44.1833,28.6500,70,Hotel Chérica
63,puerto natales,CL,-51.7236,-72.4875,57,Dorotea Patagonia Hostel
170,alexandria,EG,31.2156,29.9553,69,Smoha Zahran Haus (7th floor)
259,saint-francois,FR,46.4154,3.9054,44,Chez Lily
271,luc-la-primaube,FR,44.3144,2.5355,42,Le Bistro
273,sasolburg,ZA,-26.8136,27.8170,32,Be My Guest Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
# Configure the map plot
hotel_map = hotel_df.hvplot.points("Lng", 
                                   "Lat", 
                                   geo = True,
                                   color = "City",
                                   alpha = 0.8,
                                   size = "Humidity",
                                   tiles = "OSM",
                                   hover_cols = ["Hotel Name", "Country"]
                                  )

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)